In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import os
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
import numpy as np
import torch
from torch import nn
# from torch.utils.data import DataLoaders
from torchvision import datasets
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from fastai.vision.all import *
import gc

data_pth = Path('../input/petfinder-pawpularity-score/')

if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'
df_train = pd.read_csv(data_pth/'train.csv')
#df_train['Pawpularity'].hist(figsize = (10, 5), bins = 100)

# val_thr_lw = 17
# val_thr_hg = 55
# thr_q = 90

# df_train.sort_values(by=["Pawpularity"])

# for i in df_train["Pawpularity"]:
#     quant = df_train["Pawpularity"].value_counts()[i]
#     if ((i <= val_thr_hg) and (i >= val_thr_lw) and (quant > thr_q)) or ((i == 100) and (quant > thr_q)):
#         ind = df_train.index[df_train['Pawpularity'] == i].tolist()

#         for x in range(int(quant - thr_q)):
#             df_train.drop(ind[x], axis = 0, inplace = True)

#df_train['Pawpularity'].hist(figsize = (10, 5), bins = 100)

df_train['score_normed'] = df_train['Pawpularity']/100
df_train["path"] = df_train["Id"].map(lambda x:str(data_pth/'train'/x)+'.jpg')
            
df_train = df_train.sample(frac=1).reset_index(drop=True)

df_test_high = df_train[df_train.Pawpularity > 35]
df_test_high = df_test_high.sample(frac=1).reset_index(drop=True)

df_test_low = df_train[df_train.Pawpularity <= 35]
df_test_low = df_test_low.sample(frac=1).reset_index(drop=True)
            
dls = ImageDataLoaders.from_df(df_train, valid_pct=0.3, seed = 228, fn_col = 'path', label_col = 'score_normed', 
                               y_block = RegressionBlock, bs = 32, num_workers = 8, item_tfms = Resize(224), 
                               batch_tfms = setup_aug_tfms([Flip(), Brightness(), Contrast(), Hue(), Saturation()]))



In [ ]:
def seed(n):
    set_seed(n, reproducible=True)
    torch.manual_seed(n)
    torch.cuda.manual_seed(n)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True

def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(torch.sigmoid(input.flatten()), target))

seed(228)

In [ ]:
#model = create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=dls.c)

#learn = Learner(dls, model, loss_func=BCEWithLogitsLossFlat(), metrics=AccumMetric(petfinder_rmse)).to_fp16()

#learn.lr_find(end_lr=3e-2)

In [ ]:
train = 0
if train == 1:

    for i in range(6):
        dls = ImageDataLoaders.from_df(df_train, valid_pct=0.3, seed = 228, fn_col = 'path', label_col = 'score_normed', 
                                    y_block = RegressionBlock, bs = 32, num_workers = 8, item_tfms = Resize(224), 
                                batch_tfms = setup_aug_tfms([Flip(), Brightness(), Contrast(), Hue(), Saturation()]))

        model = create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=dls.c)
        learn = Learner(dls, model, loss_func=BCEWithLogitsLossFlat(), metrics=AccumMetric(petfinder_rmse)).to_fp16()
        learn.fit_one_cycle(10, 1e-5, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=3)])

        learn = learn.to_fp32()
        learn.export('swin_tuned_exp{i}' + str(i))

        del learn
        torch.cuda.empty_cache()
        gc.collect()


    for i in range(2):
        dls = ImageDataLoaders.from_df(df_test_high, valid_pct=0.2, seed = 228, fn_col = 'path', label_col = 'score_normed', 
                                y_block = RegressionBlock, bs = 32, num_workers = 8, item_tfms = Resize(224), 
                                batch_tfms = setup_aug_tfms([Flip(), Brightness(), Contrast(), Hue(), Saturation()]))
        
        model = create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=dls.c)

        learn = Learner(dls, model, loss_func=BCEWithLogitsLossFlat(), metrics=AccumMetric(petfinder_rmse)).to_fp16()
        learn.fit_one_cycle(10, 1e-5, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=3)])
        #learn.recorder.plot_loss()
        #plt.savefig('loss.png')

        learn = learn.to_fp32()
        learn.export('swin_high_exp' + str(i))
        
        del learn

        torch.cuda.empty_cache()

        gc.collect()
        
    for i in range(2):
        dls = ImageDataLoaders.from_df(df_test_low, valid_pct=0.2, seed = 228, fn_col = 'path', label_col = 'score_normed', 
                                y_block = RegressionBlock, bs = 32, num_workers = 8, item_tfms = Resize(224), 
                                batch_tfms = setup_aug_tfms([Flip(), Brightness(), Contrast(), Hue(), Saturation()]))
        
        model = create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=dls.c)

        learn = Learner(dls, model, loss_func=BCEWithLogitsLossFlat(), metrics=AccumMetric(petfinder_rmse)).to_fp16()
        learn.fit_one_cycle(10, 1e-5, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=3)])
        #learn.recorder.plot_loss()
        #plt.savefig('loss.png')

        learn = learn.to_fp32()
        learn.export('swin_low_exp' + str(i))
        
        del learn

        torch.cuda.empty_cache()

        gc.collect()


In [ ]:
df_test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
df_test["path"] = df_test["Id"].map(lambda x:str(data_pth/'test'/x)+'.jpg')
test_dl = dls.test_dl(df_test)

pt1 = f'/kaggle/input/swin224-tuned/swin_tuned_exp'
pt2 = f'/kaggle/input/swin224-tuned/swin_high_exp'
pt3 = f'/kaggle/input/swin224-tuned/swin_low_exp'

pred = []

for i in range(6):
    learn = load_learner((pt1 + str(i)), cpu = False)

    p, _ = learn.tta(dl=test_dl, n=5, beta=0)
    pred.append(p)
    
for i in range(2):
    learn = load_learner((pt2 + str(i)), cpu = False)

    p, _ = learn.tta(dl=test_dl, n=5, beta=0)
    pred.append(p)
    
for i in range(2):
    learn = load_learner((pt3 + str(i)), cpu = False)

    p, _ = learn.tta(dl=test_dl, n=5, beta=0)
    pred.append(p)
    
sub_df = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
sub_df['Pawpularity'] = (np.mean(np.hstack(pred), axis=1))*100    
sub_df.to_csv('submission.csv',index=False)

